### Feature Engineering and Data Processing: Y-SIGN data 2022-03
This file is for R&D of features for the model. Features should be created using functions that can then be cut and paste into the model_functions.py file to be imported into the final model .ipynb file.

We should split our data before we do any of this! (before splitting, need expand the intersections data to every road, or just use the direct adjacant values).

In [1]:
# Import 3rd party libraries
import os
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

# Configure Notebook
%matplotlib inline
plt.style.use('dark_background')
#plt.style.use('ggplot')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

#### Import the train data (LTS data + intersections, neighbourhoods ...)

#### Data Processing

Import the data

In [15]:
train_data = pd.read_csv('C:/Users/Keagan Rankin/Documents/PycharmProj/data/training_data/ysign_dropoff2_iter15_train.csv')
train_data.head()

,Unnamed: 0,OBJECTID,GEO_ID,LFN_ID,LF_NAME,FNODE,TNODE,ONE_WAY_DI,length_in_,GEO_ID_1,LTS,Shape_Leng,geometry,ysign vehicle,ysign ped,high access
0,26720,26722,1137670,3025,Bocastle Ave,13453368,13453237,0,95.530596,1137670,1,0.000883,LINESTRING (628767.3368918777 4843212.59019024...,4790.250,973.25,1
1,45540,45542,111369,8220,Kenmark Blvd,13453101,13453173,0,228.998686,111369,1,0.002671,LINESTRING (639762.0980769163 4843420.41458969...,2446.250,329.25,1
2,14362,14363,14048139,3053,Brandon Ave,13463765,14048141,0,49.852955,14048139,1,0.000607,LINESTRING (625363.2529044797 4836317.83499265...,1582.625,73.25,1
3,36418,36420,2689046,3803,Lake Shore Blvd E,13466078,13466003,0,105.443542,2689046,4,0.001222,LINESTRING (633798.3739368191 4834729.22944054...,1327.250,7.00,0
4,15812,15813,14040464,11872,Ln 1 W Clinton N Gore,14040383,14040467,0,22.935001,14040464,1,0.000214,LINESTRING (627870.1851326771 4834730.84276338...,906.375,451.50,1


In [16]:
train_data['ysign vehicle'].quantile(0.75)

5556.75

Create a function that drops outliers in the ysign data, <br>
We don't want to remove outliers based on YSIGN, but instead outliers should be removed during data cleaning before the test train split (based on the overall 8 hour peak) <br>
Even then, we don't really want to discount any measurements as these are "peaks" over time. lets not remove outliers <br>
We can however remove all the rows we don't want

In [17]:
def ysign_droprows(data, keep):
    """
    Function drops rows that are not passed in 'keep' list.
    """
    
    #Drop
    data_drop = data[keep]
    
    return data_drop

keep_rows = ['OBJECTID',	'GEO_ID',	'ONE_WAY_DI',	
             'LTS',	'geometry',	'ysign vehicle',	
             'ysign ped',	'high access']

train_data = ysign_droprows(train_data, keep_rows)
train_data.head()

,OBJECTID,GEO_ID,ONE_WAY_DI,LTS,geometry,ysign vehicle,ysign ped,high access
0,26722,1137670,0,1,LINESTRING (628767.3368918777 4843212.59019024...,4790.250,973.25,1
1,45542,111369,0,1,LINESTRING (639762.0980769163 4843420.41458969...,2446.250,329.25,1
2,14363,14048139,0,1,LINESTRING (625363.2529044797 4836317.83499265...,1582.625,73.25,1
3,36420,2689046,0,4,LINESTRING (633798.3739368191 4834729.22944054...,1327.250,7.00,0
4,15813,14040464,0,1,LINESTRING (627870.1851326771 4834730.84276338...,906.375,451.50,1


Lets scale the ysign data with a standard scaler

In [18]:
from sklearn.preprocessing import StandardScaler
def ysign_scale(data, scale_feats):
    """
    Scales the passed columns of the ysign data.
    """
    
    # Create scaler
    scaler = StandardScaler()
    scaler.fit(data[scale_feats])
    
    scaled = data[scale_feats].copy()
    
    # Convert numeric features to standard units
    scaled.iloc[:, :] = scaler.transform(scaled)
    
    data[scale_feats] = scaled
    
    return data
    
train_data = ysign_scale(train_data, ['ysign vehicle', 'ysign ped'])
train_data.head()

,OBJECTID,GEO_ID,ONE_WAY_DI,LTS,geometry,ysign vehicle,ysign ped,high access
0,26722,1137670,0,1,LINESTRING (628767.3368918777 4843212.59019024...,0.094336,0.258095,1
1,45542,111369,0,1,LINESTRING (639762.0980769163 4843420.41458969...,-0.330468,-0.118231,1
2,14363,14048139,0,1,LINESTRING (625363.2529044797 4836317.83499265...,-0.486982,-0.267827,1
3,36420,2689046,0,4,LINESTRING (633798.3739368191 4834729.22944054...,-0.533264,-0.306540,0
4,15813,14040464,0,1,LINESTRING (627870.1851326771 4834730.84276338...,-0.609539,-0.046794,1


We also suspect that there is a geospatial relationship for LTS. If we do PCA like in assignment 7 in the centroids of the centrelines, <br>
and then dummy encode regions in toronto along the principle components, we can use this as a feature.

In [ ]:
from sklearn.decomposition import PCA
def add_regions(data, components):
    """
    function extract PC of Toronto's geometry and bins it into a # of regions.
    """
    
    
    # Create the PCA function
    pca = PCA(n_components=components)
    pca.fit(data[['pickup_lon', 'pickup_lat']])
    
    print('Principal component 1 explains {:.2f} % of the variance in "lon" and "lat".'.format(pca.explained_variance_ratio_[0] * 100))
    print('Principal component 2 explains {:.2f} % of the variance in "lon" and "lat".'.format(pca.explained_variance_ratio_[1] * 100))
    
add_regions(data

#### Advanced cross validation
We may want to do an advanced, spatial cross validation due to the nature of the data.<br>
create a function that performs this cross val